In [ ]:
import os
import re
import sys
import numpy as np
import pandas as pd
import xarray as xr
from natsort import natsorted
from scipy.stats import zscore
from os.path import join as pjoin
import plotly.graph_objects as go

sys.path.append('../../')
import mannkendall as mk
import plotting_functions as pf
import circletrack_neural as ctn

In [ ]:
## Settings
project_dir = 'MultiCon_Imaging'
experiment_dir = 'MultiCon_Imaging6'
output_path = os.path.abspath(f'../../../{project_dir}/{experiment_dir}/output')
mouse_list = ['mc54', 'mc55', 'mc56', 'mc58', 'mc59', 'mc60'] 
mouse = 'mc60'
alpha = 0.001 ## for Mann-Kendall trend test

In [ ]:
## Can use this code block to check individual sessions and QC by eye units that the Mann-Kendall trend test is detecting to be dropped
session_idx = 0
c_path = pjoin(output_path, f'aligned_minian/{mouse}/C')
yra_path = pjoin(output_path, f'aligned_minian/{mouse}/YrA')
s_path = pjoin(output_path, f'aligned_minian/{mouse}/S')
place_path = pjoin(output_path, f'aligned_place_cells/{mouse}/S')

c_save = pjoin(output_path, f'aligned_minian_two/{mouse}/C')
yra_save = pjoin(output_path, f'aligned_minian_two/{mouse}/YrA')
s_save = pjoin(output_path, f'aligned_minian_two/{mouse}/S')
place_save = pjoin(output_path, f'aligned_place_cells_two/{mouse}/S')
for idx, session in enumerate(natsorted(os.listdir(c_path))):
    if idx == session_idx:
        day = re.search('_([0-9]+)', session)[0]
        fig = pf.custom_graph_template(x_title='Time (s)', y_title='Activity (a.u.)', width=700)
        cell_trends = {'unit_id': [], 'trend': [], 'mk_score': [], 'pval': []}

        ## Load data
        C = xr.open_dataset(pjoin(c_path, session))['C']
        if len(day) == 2:
            yra_string = f'{mouse}_YrA_{day[-1]}.nc'
            s_string = f'{mouse}_S_{day[-1]}.nc'
            c_string = f'{mouse}_C_{day[-1]}.nc'
        else:
            yra_string = f'{mouse}_YrA_{day[-2:]}.nc'
            s_string = f'{mouse}_S_{day[-2:]}.nc'
            c_string = f'{mouse}_C_{day[-2:]}.nc'
        YrA = xr.open_dataset(pjoin(yra_path, yra_string))['YrA']
        S = xr.open_dataset(pjoin(s_path, s_string))['S']
        S_place = xr.open_dataset(pjoin(place_path, s_string))['S']

        ## Mann-Kendall trend test on taking the minimum value in a 20-second bin of z-scored data
        ## Rationale is that if there is an increasing/decreasing trend, the z-scored values get brought above or
        ## below zero, and Mann-Kendall can detect this. 20-second bin allows for less false positives
        cell_act = C.values
        zdata = zscore(cell_act, axis=1)
        bin_act = ctn.bin_activity(zdata, bin_size_seconds=20, func=np.min)
        for uid in np.arange(0, C.shape[0]):
            mkt = mk.original_test(bin_act[uid, :])
            cell_trends['unit_id'].append(uid)
            cell_trends['trend'].append(mkt.trend)
            cell_trends['mk_score'].append(np.abs(mkt.s))
            cell_trends['pval'].append(mkt.p)
        trends = pd.DataFrame(cell_trends)
        dropped_uids = trends[(trends['pval'] < alpha)].reset_index(drop=True)

        for neuron in dropped_uids['unit_id']:
            fig.add_trace(go.Scattergl(x=YrA['behav_t'].values, y=YrA.values[neuron, :], mode='lines', visible='legendonly',
                                    line_color='black', name=f'Neuron {neuron}', showlegend=False, legendgroup=f'{neuron}'))
            fig.add_trace(go.Scattergl(x=C['behav_t'].values, y=zdata[neuron, :], mode='lines', visible='legendonly',
                                    line_color='red', name=f'Neuron {neuron}', legendgroup=f'{neuron}'))
    else:
        pass
fig.show()

In [ ]:
## Alter boolean to keep cells you don't want to drop
## The Neuron number on the graph corresponds to the index
keep_cell_ids = []
to_be_dropped = (trends['pval'] < alpha).to_numpy()
if dropped_uids.shape[0] == 0:
    print('All units passed quality control!')
elif len(keep_cell_ids) == 0:
    print('Dropping all identified units!')
else:
    print('Keeping selected units...')
    for val in keep_cell_ids:
        to_be_dropped[val] = False

In [ ]:
## Assign boolean array as a coordinate to each activity array
C = C.assign_coords(passed_qc=('unit_id', ~to_be_dropped))
S = S.assign_coords(passed_qc=('unit_id', ~to_be_dropped))
YrA = YrA.assign_coords(passed_qc=('unit_id', ~to_be_dropped))
S_place = S_place.assign_coords(passed_qc=('unit_id', ~to_be_dropped))

## Save output
if not os.path.exists(c_save):
    os.makedirs(c_save)

if not os.path.exists(s_save):
    os.makedirs(s_save)

if not os.path.exists(yra_save):
    os.makedirs(yra_save)

if not os.path.exists(place_save):
    os.makedirs(place_save)
    
C.to_netcdf(pjoin(c_save, c_string))
S.to_netcdf(pjoin(s_save, s_string))
YrA.to_netcdf(pjoin(yra_save, yra_string))
S_place.to_netcdf(pjoin(place_save, s_string))